## Secure Hashing Algorithm - SHA-512

In [14]:
import hashlib

text = 'abc'

m = hashlib.sha512(text.encode('UTF-8'))
print(m.hexdigest())

ddaf35a193617abacc417349ae20413112e6fa4e89a97ea20a9eeee64b55d39a2192992a274fc1a836ba3c23a3feebbd454d4423643ce80e2a9ac94fa54ca49f


In [15]:
import numpy as np
import pandas as pd
import string
import random

In [16]:
# 8 thanh ghi 64 bits
initial_hash = (
    0x6a09e667f3bcc908,
    0xbb67ae8584caa73b,
    0x3c6ef372fe94f82b,
    0xa54ff53a5f1d36f1,
    0x510e527fade682d1,
    0x9b05688c2b3e6c1f,
    0x1f83d9abfb41bd6b,
    0x5be0cd19137e2179,
)

# K_t (0 <= t <= 79)
# Khoá mỗi vòng
K = (
    0x428a2f98d728ae22, 0x7137449123ef65cd,
    0xb5c0fbcfec4d3b2f, 0xe9b5dba58189dbbc,
    0x3956c25bf348b538, 0x59f111f1b605d019,
    0x923f82a4af194f9b, 0xab1c5ed5da6d8118,
    0xd807aa98a3030242, 0x12835b0145706fbe,
    0x243185be4ee4b28c, 0x550c7dc3d5ffb4e2,
    0x72be5d74f27b896f, 0x80deb1fe3b1696b1,
    0x9bdc06a725c71235, 0xc19bf174cf692694,
    0xe49b69c19ef14ad2, 0xefbe4786384f25e3,
    0x0fc19dc68b8cd5b5, 0x240ca1cc77ac9c65,
    0x2de92c6f592b0275, 0x4a7484aa6ea6e483,
    0x5cb0a9dcbd41fbd4, 0x76f988da831153b5,
    0x983e5152ee66dfab, 0xa831c66d2db43210,
    0xb00327c898fb213f, 0xbf597fc7beef0ee4,
    0xc6e00bf33da88fc2, 0xd5a79147930aa725,
    0x06ca6351e003826f, 0x142929670a0e6e70,
    0x27b70a8546d22ffc, 0x2e1b21385c26c926,
    0x4d2c6dfc5ac42aed, 0x53380d139d95b3df,
    0x650a73548baf63de, 0x766a0abb3c77b2a8,
    0x81c2c92e47edaee6, 0x92722c851482353b,
    0xa2bfe8a14cf10364, 0xa81a664bbc423001,
    0xc24b8b70d0f89791, 0xc76c51a30654be30,
    0xd192e819d6ef5218, 0xd69906245565a910,
    0xf40e35855771202a, 0x106aa07032bbd1b8,
    0x19a4c116b8d2d0c8, 0x1e376c085141ab53,
    0x2748774cdf8eeb99, 0x34b0bcb5e19b48a8,
    0x391c0cb3c5c95a63, 0x4ed8aa4ae3418acb,
    0x5b9cca4f7763e373, 0x682e6ff3d6b2b8a3,
    0x748f82ee5defb2fc, 0x78a5636f43172f60,
    0x84c87814a1f0ab72, 0x8cc702081a6439ec,
    0x90befffa23631e28, 0xa4506cebde82bde9,
    0xbef9a3f7b2c67915, 0xc67178f2e372532b,
    0xca273eceea26619c, 0xd186b8c721c0c207,
    0xeada7dd6cde0eb1e, 0xf57d4f7fee6ed178,
    0x06f067aa72176fba, 0x0a637dc5a2c898a6,
    0x113f9804bef90dae, 0x1b710b35131c471b,
    0x28db77f523047d84, 0x32caab7b40c72493,
    0x3c9ebe0a15c9bebc, 0x431d67c49c100d4c,
    0x4cc5d4becb3e42b6, 0x597f299cfc657e2a,
    0x5fcb6fab3ad6faec, 0x6c44198c4a475817
)

In [17]:
class SHA512(object):
    def __init__ (self):
        self.h = list(initial_hash)
        self.h_hex = None
        self.message_bin = None
        self.length = 0

    def rotate_right(self, n, bits):
        return (n >> bits) | (n << (64-bits))
    
    def pad(self):
        # padding to 896 mod 1024
        padding_len = (896 - (self.length%1024) + 1023)%1024
        self.message_bin += "1"
        self.message_bin += "0"*padding_len

        # padding length info
        self.message_bin += '{0:b}'.format(self.length).zfill(128)
        return self.message_bin

    def process(self):
        # Get list 1024 bits block
        blocks = [self.message_bin[i:i+1024] for i in range(0, len(self.message_bin), 1024)]
        # Process each 1024 bits block in 80 rounds
        for block in blocks:
            # break block into sixteen 64-bit words (W[0]-W[15]) (in integer)
            W = [int(block[i:i+64],2) for i in range(0, len(block), 64)]
            # 64 last rounds
            for i in range(16, 80):
                # ROTR(x,1) ^ ROTR(x,8) ^ SHR(x,7) | x = W[i-15]
                s0 = self.rotate_right(W[i-15], 1) ^ self.rotate_right(W[i-15], 8) ^ (W[i-15] >> 7)
                # ROTR(x,19) ^ ROTR(x,61) ^ SHR(x,7) | x = W[i-2]
                s1 = self.rotate_right(W[i-2], 19) ^ self.rotate_right(W[i-2], 61) ^ (W[i-2] >> 6)
                # W[t] = s1(W[t-2]) + W[t-7] + s0(W[t-15]) + W[t-16]
                W.append((W[i-16] + s0 + W[i-7] + s1)%2**64)

            # initialize working variables
            a, b, c, d, e, f, g, h = self.h # (IV)
            for i in range(80):
                # ROTR(e,14) ^ ROTR(e,18) ^ ROTR(e,41)
                s1 = self.rotate_right(e, 14) ^ self.rotate_right(e, 18) ^ self.rotate_right(e, 41)
                # ch(e,f,g) = (e&f) ^ (!f & g)
                ch = (e & f) ^ (~e & g)
                # T1 = h + ch(e,f,g) + s1(e) + W[t] + K[t]
                temp1 = (h + ch + s1 + W[i] + K[i])%2**64
                # ROTR(a,28) ^ ROTR(a,34) ^ ROTR(a,39)
                s0 = self.rotate_right(a, 28) ^ self.rotate_right(a, 34) ^ self.rotate_right(a, 39)
                # Maj(a,b,c) = (a&b) ^ (a&c) ^ (b&c)
                maj = (a & b) ^ (a & c) ^ (b & c)
                # T2 = s0(a) + Maj(a,b,c)
                temp2 = (s0 + maj)%2**64

                h = g
                g = f
                f = e
                e = (d + temp1) % 2**64
                d = c
                c = b
                b = a
                a = (temp1 + temp2) % 2**64

            # compute intermediate hash value
            self.h[0] = (self.h[0] + a) % 2**64
            self.h[1] = (self.h[1] + b) % 2**64
            self.h[2] = (self.h[2] + c) % 2**64
            self.h[3] = (self.h[3] + d) % 2**64
            self.h[4] = (self.h[4] + e) % 2**64
            self.h[5] = (self.h[5] + f) % 2**64
            self.h[6] = (self.h[6] + g) % 2**64
            self.h[7] = (self.h[7] + h) % 2**64

    def hash(self, message):
        print("Message:", message)
        self.message_bin = ''.join(['{0:b}'.format(ord(message[i])).zfill(8) for i in range(len(message))])
        print("Binary form of message:", self.message_bin)
        self.length = len(self.message_bin)
        self.pad()
        print(self.pad())
        print("Binary form after padding:", self.message_bin)
        self.process()
        print("Final digestin 512 in decimal:", self.h)
        self.h_hex = ''.join(['{0:x}'.format(i).zfill(16) for i in self.h])
        print("Final digestin 512 in hex:", self.h_hex)

In [18]:
sha = SHA512()
sha.hash("abc")

Message: abc
Binary form of message: 011000010110001001100011
01100001011000100110001110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [19]:
message = input("Enter the message: ")
sha = SHA512()
sha.hash(message)

Message: abc
Binary form of message: 011000010110001001100011
01100001011000100110001110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000